In [36]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pylab as plt
from math import ceil
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Input, LeakyReLU,Dense,BatchNormalization, Activation, Reshape, Conv2D,Conv2DTranspose,Flatten,UpSampling2D,Convolution2D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import initializers

In [84]:
# from tensorflow.python.keras.layers.convolutional import Convolution2D
# def build_cifar10_generator(ngf=64, z_dim=128):
#   init = initializers.RandomNormal(stddev=0.02)
#   G = Sequential([
#       Input(shape=(z_dim,)),
#       Dense(2*2*ngf*8,use_bias=True,kernel_constraint=init),
#       Reshape((2,2,ngf*8)),
#       BatchNormalization(),
#       LeakyReLU(.2),
#       Conv2DTranspose(ngf*4,kernel_size=5,strides=2,kernel_initializer=init),
#       BatchNormalization(),
#       LeakyReLU(0.2),
#       UpSampling2D(size=(2,2)),
#       # Convolution2D(ngf*4,5,5,padding="same"),
#       Conv2DTranspose(ngf,kernel_size=5,padding="same",
#                       use_bias=True,kernel_regularizer=init),
#       BatchNormalization(),
#       LeakyReLU(.2)
      
#   ])
#   return G

def build_cifar10_generator(ngf=64, z_dim=128):
    """ Builds CIFAR10 DCGAN Generator Model
    PARAMS
    ------
    ngf: number of generator filters
    z_dim: number of dimensions in latent vector
    RETURN
    ------
    G: keras sequential
    """
    init = initializers.RandomNormal(stddev=0.02)
    G = Sequential()
    # Dense 1: 2x2x512
    G.add(Dense(2*2*ngf*8, input_shape=(z_dim, ),
          use_bias=True, kernel_initializer=init))
    G.add(Reshape((2, 2, ngf*8)))
    G.add(BatchNormalization())
    G.add(LeakyReLU(0.2))

        # Conv 1: 4x4x256
    G.add(Conv2DTranspose(ngf*4, kernel_size=5, strides=2, 
          padding='same',use_bias=True,   
          kernel_initializer=init))
    G.add(BatchNormalization())
    G.add(LeakyReLU(0.2))

    G.add(UpSampling2D(size=(2, 2)))
    # G.add(Conv2D(ngf*4, 5, 5, padding="SAME"))

        # Conv 2: 8x8x128
    G.add(Conv2DTranspose(ngf*2, kernel_size=5, strides=2, padding='same',
                          use_bias=True, kernel_initializer=init))
    G.add(BatchNormalization())
    G.add(LeakyReLU(0.2))

    # Conv 3: 16x16x64
    G.add(Conv2DTranspose(ngf, kernel_size=5, strides=2, padding='same',
                          use_bias=True, kernel_initializer=init))
    G.add(BatchNormalization())
    G.add(LeakyReLU(0.2))

    return G

In [85]:
build_cifar10_generator(ngf=64, z_dim=128).summary()

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 2048)              264192    
                                                                 
 reshape_31 (Reshape)        (None, 2, 2, 512)         0         
                                                                 
 batch_normalization_68 (Bat  (None, 2, 2, 512)        2048      
 chNormalization)                                                
                                                                 
 leaky_re_lu_68 (LeakyReLU)  (None, 2, 2, 512)         0         
                                                                 
 conv2d_transpose_39 (Conv2D  (None, 4, 4, 256)        3277056   
 Transpose)                                                      
                                                                 
 batch_normalization_69 (Bat  (None, 4, 4, 256)      

In [101]:
def build_cifar10_discriminator(ndf=64, image_shape=(32, 32, 3)):

  init = initializers.RandomNormal(stddev=.02)

  D = Sequential()
  D.add(Conv2D(ndf,kernel_size=5, strides=2,padding="same",use_bias=True,
               kernel_initializer=init,input_shape=image_shape))
  D.add(LeakyReLU(.2))

  D.add(Conv2D(ndf*2,kernel_size=5,strides=2,padding="same",
               use_bias=True,kernel_initializer=init))
  D.add(BatchNormalization())
  D.add(LeakyReLU(.2))

  D.add(Conv2D(ndf*4,kernel_size=5,strides=2,padding="same",
               use_bias=True,kernel_initializer=init))
  D.add(BatchNormalization())
  D.add(LeakyReLU(.2))

  D.add(Conv2D(ndf*8,kernel_size=5,strides=2,padding="same",
               use_bias=True,kernel_initializer=init))
  D.add(BatchNormalization())
  D.add(LeakyReLU(.2))

  D.add(Flatten())
  D.add(Dense(1,kernel_initializer=init))
  D.add(Activation("sigmoid"))
  return D

In [102]:
build_cifar10_discriminator().summary()

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 16, 16, 64)        4864      
                                                                 
 leaky_re_lu_92 (LeakyReLU)  (None, 16, 16, 64)        0         
                                                                 
 conv2d_31 (Conv2D)          (None, 8, 8, 128)         204928    
                                                                 
 batch_normalization_85 (Bat  (None, 8, 8, 128)        512       
 chNormalization)                                                
                                                                 
 leaky_re_lu_93 (LeakyReLU)  (None, 8, 8, 128)         0         
                                                                 
 conv2d_32 (Conv2D)          (None, 4, 4, 256)         819456    
                                                     

In [103]:
def get_data():
    # load cifar10 data
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    
    # convert train and test data to float32
    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)
    
    # scale train and test data to [-1, 1]
    X_train = (X_train / 255) * 2 - 1
    X_test = (X_train / 255) * 2 - 1

    return X_train, X_test

In [104]:
def plot_images(images, filename):
    h, w, c = images.shape[1:]
    grid_size = ceil(np.sqrt(images.shape[0]))
    images = (images.reshape(grid_size, grid_size, h, w, c)
              .transpose(0, 2, 1, 3, 4)
              .reshape(grid_size*h, grid_size*w, c))
    plt.figure(figsize=(16, 16))
    plt.imsave(filename, images)

In [105]:
def plot_losses(losses_d, losses_g, filename): 
    fig, axes = plt.subplots(1, 2, figsize=(8, 2))
    axes[0].plot(losses_d)
    axes[1].plot(losses_g)
    axes[0].set_title("losses_d")
    axes[1].set_title("losses_g")
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()